## Part 1

In [11]:
from enum import Enum

In [237]:
TEST_INFILE = "inputs/day_18_test_1.txt"
INFILE = "inputs/day_18_1.txt"

#with open(TEST_INFILE) as infile:
with open(INFILE) as infile:
    lines = infile.read().splitlines()

instructions = [line.split() for line in lines]
instructions = [(i[0], int(i[1]), i[2].replace("(", "").replace(")", "")) for i in instructions]

In [238]:
instructions

[('L', 7, '#1dc5f0'),
 ('U', 6, '#4497e3'),
 ('L', 3, '#3b21c0'),
 ('U', 3, '#3d8523'),
 ('L', 9, '#58e7b2'),
 ('U', 8, '#35d213'),
 ('L', 10, '#329390'),
 ('D', 3, '#1302b3'),
 ('R', 7, '#07f802'),
 ('D', 8, '#38fbd3'),
 ('L', 7, '#6d0e72'),
 ('D', 6, '#2c2493'),
 ('L', 5, '#38b7d0'),
 ('U', 6, '#2ab7b3'),
 ('L', 2, '#0ce560'),
 ('U', 6, '#2ab7b1'),
 ('L', 2, '#2f6940'),
 ('U', 2, '#2a5a83'),
 ('L', 3, '#477150'),
 ('D', 11, '#0baea3'),
 ('L', 4, '#3899c0'),
 ('D', 3, '#34a803'),
 ('L', 5, '#55c0d0'),
 ('U', 9, '#328483'),
 ('L', 2, '#460290'),
 ('U', 5, '#48cfd3'),
 ('L', 4, '#1e3150'),
 ('U', 5, '#3b7ea3'),
 ('L', 7, '#2c0a10'),
 ('U', 11, '#51d803'),
 ('L', 4, '#39fe80'),
 ('U', 3, '#51d801'),
 ('R', 10, '#2938e0'),
 ('U', 2, '#1c17f3'),
 ('R', 4, '#0df9a0'),
 ('U', 3, '#579691'),
 ('L', 8, '#354a50'),
 ('U', 4, '#3d0131'),
 ('L', 6, '#1029b0'),
 ('U', 7, '#0c19b3'),
 ('L', 3, '#5f48c0'),
 ('U', 2, '#0c19b1'),
 ('L', 4, '#15ff10'),
 ('D', 9, '#0f08b1'),
 ('L', 5, '#1c8160'),
 ('U',

In [239]:
class Point:
    def __init__(self, row, col):
        self.row = row
        self.col = col

    def __repr__(self):
        return f"({self.row}, {self.col})"

    def __add__(self, other):
        return Point(self.row + other.row, self.col + other.col)
    
    def __sub__(self, other):
        return Point(self.row - other.row, self.col - other.col)
    
    def __mul__(self, number):
        return Point(self.row * number, self.col * number)

    def __radd__(self, other):
        return self + other
    
    def __eq__(self, other):
        return self.row == other.row and self.col == other.col
    
    def __hash__(self):
        return hash((self.row, self.col))
    

class Direction(Enum):
    UP = Point(-1, 0)
    DOWN = Point(1, 0)
    LEFT = Point(0, -1)
    RIGHT = Point(0, 1)


assert Point(0, 0) == Point(0, 0)
assert Point(1, 0) + Point(1, 10) == Point(2, 10)
p = Point(1, 0)
p += Point(1, 10)
assert p == Point(2, 10)
assert Point(1, 2) * 10 == Point(10, 20)
assert Point(20, 10) - Point(5, 5) == Point(15, 5)


class Vector:
    def __init__(self, point, direction):
        self.point = point
        self.direction = direction

    def __repr__(self):
        return f"({self.point.row}, {self.point.col}) => {self.direction}"
    
    def __eq__(self, other):
        return self.point == other.point and self.direction == other.direction
    
    def __lt__(self, other):
        return self.point.row < other.point.row or (self.point.row == other.point.row and self.point.col < other.point.col)
    
    def __hash__(self):
        return hash((self.point.row, self.point.col, self.direction))
    

assert Vector(Point(0, 0), Direction.UP) == Vector(Point(0, 0), Direction.UP)

In [240]:
START = Point(0, 0)
edges = {}
pos = START
for direction, steps, color in instructions:
    #print(f"Instruction: {direction} {steps} {color}")
    match direction:
        case "R":
            dir = Direction.RIGHT.value
        case "L":
            dir = Direction.LEFT.value
        case "U":
            dir = Direction.UP.value
        case "D":
            dir = Direction.DOWN.value

    for _ in range(steps):
        last_pos = pos
        pos += dir
        edges[last_pos] = pos
    

In [241]:
len(edges)

3890

In [242]:
boundary = set(edges.keys()).union(set(edges.values()))
min_row, max_row = min([p.row for p in boundary]), max([p.row for p in boundary])
min_col, max_col = min([p.col for p in boundary]), max([p.col for p in boundary])
print(f"min=({min_row}, {min_col}), max=({max_row}, {max_col})")

min=(-191, -125), max=(122, 290)


In [243]:
for row_n in range(min_row, max_row + 1):
    for col_n in range(min_col, max_col + 1):
        if Point(row_n, col_n) in edges:
            print("#", end="")
        else:
            print(".", end="")
    print()

.............................................................................................................................................####..............................................................................................................................................................#####............................................................................................................
.............................................................................................................................................#..#..............................................................................................................................................................#...#............................................................................................................
.............................................................................................................................................#..#.....................

In [244]:
# pick one inside point to flood fill from
#INSIDE_POINT = Point(13, 5)
INSIDE_POINT = Point(1, 4)
queue = [INSIDE_POINT]
inside = set([INSIDE_POINT])
visited = set()
while len(queue) > 0:
    point = queue.pop(0)
    if point not in visited:
        if (point + Direction.UP.value) not in boundary:
            inside.add(point + Direction.UP.value)
            queue.append(point + Direction.UP.value)
        if (point + Direction.DOWN.value) not in boundary:
            inside.add(point + Direction.DOWN.value)
            queue.append(point + Direction.DOWN.value)  
        if (point + Direction.LEFT.value) not in boundary:
            inside.add(point + Direction.LEFT.value)
            queue.append(point + Direction.LEFT.value)
        if (point + Direction.RIGHT.value) not in boundary:
            inside.add(point + Direction.RIGHT.value)
            queue.append(point + Direction.RIGHT.value)
        visited.add(point)

In [245]:
for row_n in range(min_row, max_row + 1):
    for col_n in range(min_col, max_col + 1):
        if Point(row_n, col_n) in edges or Point(row_n, col_n) in inside:
            print("#", end="")
        else:
            print(".", end="")
    print()

.............................................................................................................................................####..............................................................................................................................................................#####............................................................................................................
.............................................................................................................................................####..............................................................................................................................................................#####............................................................................................................
.............................................................................................................................................####.....................

In [246]:
len(inside.union(boundary))

49061

## Part 2

In [247]:
from itertools import pairwise

In [248]:

instructions

[('L', 7, '#1dc5f0'),
 ('U', 6, '#4497e3'),
 ('L', 3, '#3b21c0'),
 ('U', 3, '#3d8523'),
 ('L', 9, '#58e7b2'),
 ('U', 8, '#35d213'),
 ('L', 10, '#329390'),
 ('D', 3, '#1302b3'),
 ('R', 7, '#07f802'),
 ('D', 8, '#38fbd3'),
 ('L', 7, '#6d0e72'),
 ('D', 6, '#2c2493'),
 ('L', 5, '#38b7d0'),
 ('U', 6, '#2ab7b3'),
 ('L', 2, '#0ce560'),
 ('U', 6, '#2ab7b1'),
 ('L', 2, '#2f6940'),
 ('U', 2, '#2a5a83'),
 ('L', 3, '#477150'),
 ('D', 11, '#0baea3'),
 ('L', 4, '#3899c0'),
 ('D', 3, '#34a803'),
 ('L', 5, '#55c0d0'),
 ('U', 9, '#328483'),
 ('L', 2, '#460290'),
 ('U', 5, '#48cfd3'),
 ('L', 4, '#1e3150'),
 ('U', 5, '#3b7ea3'),
 ('L', 7, '#2c0a10'),
 ('U', 11, '#51d803'),
 ('L', 4, '#39fe80'),
 ('U', 3, '#51d801'),
 ('R', 10, '#2938e0'),
 ('U', 2, '#1c17f3'),
 ('R', 4, '#0df9a0'),
 ('U', 3, '#579691'),
 ('L', 8, '#354a50'),
 ('U', 4, '#3d0131'),
 ('L', 6, '#1029b0'),
 ('U', 7, '#0c19b3'),
 ('L', 3, '#5f48c0'),
 ('U', 2, '#0c19b1'),
 ('L', 4, '#15ff10'),
 ('D', 9, '#0f08b1'),
 ('L', 5, '#1c8160'),
 ('U',

In [249]:
START = Point(0, 0)
vertices = [START]
pos = START
for direction, steps, color in instructions:
    print(f"Instruction: {direction} {steps} {color}")
    
    steps = int(color[1:6], 16)

    match color[6]:
        case "0":
            dir = Direction.RIGHT.value
        case "1":
            dir = Direction.DOWN.value
        case "2":
            dir = Direction.LEFT.value
        case "3":
            dir = Direction.UP.value
        
    pos += dir * steps
    vertices.append(pos)

Instruction: L 7 #1dc5f0
Instruction: U 6 #4497e3
Instruction: L 3 #3b21c0
Instruction: U 3 #3d8523
Instruction: L 9 #58e7b2
Instruction: U 8 #35d213
Instruction: L 10 #329390
Instruction: D 3 #1302b3
Instruction: R 7 #07f802
Instruction: D 8 #38fbd3
Instruction: L 7 #6d0e72
Instruction: D 6 #2c2493
Instruction: L 5 #38b7d0
Instruction: U 6 #2ab7b3
Instruction: L 2 #0ce560
Instruction: U 6 #2ab7b1
Instruction: L 2 #2f6940
Instruction: U 2 #2a5a83
Instruction: L 3 #477150
Instruction: D 11 #0baea3
Instruction: L 4 #3899c0
Instruction: D 3 #34a803
Instruction: L 5 #55c0d0
Instruction: U 9 #328483
Instruction: L 2 #460290
Instruction: U 5 #48cfd3
Instruction: L 4 #1e3150
Instruction: U 5 #3b7ea3
Instruction: L 7 #2c0a10
Instruction: U 11 #51d803
Instruction: L 4 #39fe80
Instruction: U 3 #51d801
Instruction: R 10 #2938e0
Instruction: U 2 #1c17f3
Instruction: R 4 #0df9a0
Instruction: U 3 #579691
Instruction: L 8 #354a50
Instruction: U 4 #3d0131
Instruction: L 6 #1029b0
Instruction: U 7 #0c1

In [250]:
vertices

[(0, 0),
 (0, 121951),
 (-280958, 121951),
 (-280958, 364155),
 (-532944, 364155),
 (-532944, 0),
 (-753393, 0),
 (-753393, 207161),
 (-831260, 207161),
 (-831260, 174521),
 (-1064665, 174521),
 (-1064665, -272174),
 (-1245474, -272174),
 (-1245474, -39857),
 (-1420445, -39857),
 (-1420445, 12965),
 (-1245474, 12965),
 (-1245474, 207161),
 (-1418954, 207161),
 (-1418954, 499790),
 (-1466804, 499790),
 (-1466804, 731626),
 (-1682484, 731626),
 (-1682484, 1082871),
 (-1889404, 1082871),
 (-1889404, 1369632),
 (-2187641, 1369632),
 (-2187641, 1493301),
 (-2431331, 1493301),
 (-2431331, 1673686),
 (-2766563, 1673686),
 (-2766563, 1911230),
 (-2431331, 1911230),
 (-2431331, 2080076),
 (-2546402, 2080076),
 (-2546402, 2137318),
 (-2187641, 2137318),
 (-2187641, 2355595),
 (-1937766, 2355595),
 (-1937766, 2421798),
 (-1987329, 2421798),
 (-1987329, 2812082),
 (-1937766, 2812082),
 (-1937766, 2902179),
 (-1876187, 2902179),
 (-1876187, 3018937),
 (-1518884, 3018937),
 (-1518884, 3204679),
 (-1

In [251]:
def cross_product(p1, p2):
    return p1.row * p2.col - p1.col * p2.row


def get_area(vertices):
    area = 0
    for p1, p2 in pairwise(vertices):
        area += cross_product(p1, p2)
        #print(f"{p1}, {p2} => {cross_product(p1, p2)}")

    return abs(area) / 2


def get_boundary(vertices):
    b = 0
    for p1, p2 in pairwise(vertices):
        if p1.row == p2.row:
            b += abs(p2.col - p1.col)
        else:
            b += abs(p2.row - p1.row)

    return b


def get_inside_via_picks(vertices):
    A = get_area(vertices)
    b = get_boundary(vertices)
    return A - b / 2 + 1
        

In [252]:
b = get_boundary(vertices)
i =get_inside_via_picks(vertices)
i + b

92556825427032.0